# Mangrove in Cambodia

*Written by Men Vuthy, 2022*

---

\
These data are retrieved from the dataset of the Global Mangrove Watch (GMW) version 3.0. Ref: https://www.mdpi.com/2072-4292/14/15/3657

[![Github repo](https://img.shields.io/badge/GitHub-Repository-red.svg)](https://github.com/menvuthy/Mangrove-in-Cambodia)
[![DOI](https://img.shields.io/badge/DOI-10.5281/zenodo.6894273-blue.svg)](https://doi.org/10.3390/rs14153657)



*Mount Google Drive*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

*Set path to local drive directory*

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/Other topic/Mangrove in Cambodia

*Install and import modules*

In [ ]:
!pip install rasterio
!pip install contextily

In [2]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import rasterio
from rasterio.plot import show
import contextily as ctx

<div class="markdown-google-sans">

## **1. Read datasource**
</div>


In [ ]:
# File and folder paths
file_path = "data"

# Make a search criteria to select the ndvi files
q = os.path.join(file_path, "*.tif")

# sorted files by name
paths = sorted(glob.glob(q)) 

# Open all files in folder
image_source, image_array = [], []

# Create a list of filepath
for file in paths:
    image = rasterio.open(file)
    array = image.read(1)
    image_source.append(image)
    image_array.append(array)
image_source

<div class="markdown-google-sans">

## **2. Calculate Mangrove Period of Existence from 1996 to 2020**
</div>

*Calculate Mangrove Change over 20 years*

In [4]:
# Create list of year
year_list = []
for i in paths:
  year_list.append(int(i[23:-4]))

# Create number of years
i = 0
num_year = []
while i < 10:
    if i == 0:
      num_year.append(1)
    num_year.append((year_list[i+1] - year_list[i]))
    if i == 10:
        num_year.append(1)
        break
    i += 1

# Create new array with number of year
new_image_array = []
for i in range(len(image_array)):
  array = np.where(image_array[i] != 0, num_year[i], image_array[i])
  new_image_array.append(array)

# Create mangrove change from 1996 to 2020
mangrove_change = sum(new_image_array)

*Create plot for visualizing output data*

In [ ]:
# Create plot
fig = plt.subplots(figsize=(10, 10))
plt.imshow(mangrove_change, plt.cm.jet)
plt.title('Mangrove Change in Cambodia from 1996 to 2020', fontsize=15)
plt.show();

In [ ]:
# Create plot of annual mangrove
for i in range(len(image_array)):
  fig = plt.subplots(figsize=(15, 10))
  plt.imshow(image_array[i], plt.cm.jet)
  plt.title('Mangrove in Cambodia - '+paths[i][23:-4], fontsize=15)
  plt.savefig('image/mangrove_cambodia_'+paths[i][23:-4]+'.png', dpi=300)
  plt.clf()

*Export output of Mangrove change to image in geotif (.tif)*

In [ ]:
# Copy the metadata
meta = image_source[0].meta.copy()

# outputh path
out_fp = "result/"

# Output raster
output = os.path.join(out_fp, "mangrove_change_96_20.tif")

# Update the metadata
meta.update({'driver': 'GTiff',
                 'dtype': 'uint8',
                 'nodata': 0,
                 'width': mangrove_change.shape[1],
                 'height': mangrove_change.shape[0],
                 'crs': image_source[0].crs,
                 'count':1,
                 'transform': image_source[0].transform
                })
# Write image
with rasterio.open(output, "w", **meta) as dest:
    dest.write(mangrove_change.astype(np.uint8), indexes=1)

<div class="markdown-google-sans">

## **3. Calculate Loss and Gain of Mangrove from 1996 to 2020**
</div>

*Calculate loss and gain*

In [5]:
# Calculate Loss
loss = image_array[0] - image_array[10]
loss = np.where(loss == 255, 0, loss)

# Calculate Gain
gain = image_array[10] - image_array[0]
gain = np.where(gain == 255, 0, gain)

*Export loss mangrove to image in geotiff (.tif)*

In [ ]:
# Copy the metadata
meta = image_source[0].meta.copy()

# outputh path
out_fp = "result/"

# Output raster
output = os.path.join(out_fp, "mangrove_loss_96_20.tif")

# Update the metadata
meta.update({'driver': 'GTiff',
                 'dtype': 'uint8',
                 'nodata': 0,
                 'width': loss.shape[1],
                 'height': loss.shape[0],
                 'crs': image_source[0].crs,
                 'count':1,
                 'transform': image_source[0].transform
                })
# Write image
with rasterio.open(output, "w", **meta) as dest:
    dest.write(loss.astype(np.uint8), indexes=1)

*Export gain mangrove to image in geotiff (.tif)*

In [ ]:
# Copy the metadata
meta = image_source[0].meta.copy()

# outputh path
out_fp = "result/"

# Output raster
output = os.path.join(out_fp, "mangrove_gain_96_20.tif")

# Update the metadata
meta.update({'driver': 'GTiff',
                 'dtype': 'uint8',
                 'nodata': 0,
                 'width': gain.shape[1],
                 'height': gain.shape[0],
                 'crs': image_source[0].crs,
                 'count':1,
                 'transform': image_source[0].transform
                })
# Write image
with rasterio.open(output, "w", **meta) as dest:
    dest.write(gain.astype(np.uint8), indexes=1)

<div class="markdown-google-sans">

## **4. Visualization**
</div>

In [ ]:
# Parameter
transform = image_source[0].transform

# Create plot
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(20,7))
# use imshow so that we have something to map the colorbar to
image_hidden = ax1.imshow(mangrove_change, 
                         cmap=plt.cm.jet, 
                         vmin=0, 
                         vmax=25)
divider = make_axes_locatable(ax1)
cax = divider.append_axes('right', size='3%', pad=0.1)

# Mangrove change
show(mangrove_change, ax=ax1, cmap=plt.cm.jet, transform=transform, title='Mangrove Change')
ax1.set_xlim(102.85, 103.2)
ax1.set_ylim(11.4, 11.7)
ax1.set_ylabel('Longitude')
ax1.set_xlabel('Latitude')
# Mangrove loss
show(loss, ax=ax2, cmap=plt.cm.jet, transform=transform, title='Mangrove Loss')
ax2.set_xlim(102.85, 103.2)
ax2.set_ylim(11.4, 11.7)
ax2.set_ylabel('Longitude')
ax2.set_xlabel('Latitude')

# Mangrove gain
show(gain, ax=ax3, cmap=plt.cm.jet, transform=transform, title='Mangrove Gain')
ax3.set_xlim(102.85, 103.2)
ax3.set_ylim(11.4, 11.7)
ax3.set_ylabel('Longitude')
ax3.set_xlabel('Latitude')

# add colorbar using the now hidden image
fig.colorbar(image_hidden, cax=cax)

plt.tight_layout()
plt.savefig('image/image-2.png', dpi=300)